In [1]:
import qsharp

Preparing Q# environment...


In [ ]:
%%qsharp
open Microsoft.Quantum.Measurement;
open Microsoft.Quantum.Arrays;
open Microsoft.Quantum.Convert;
open Microsoft.Quantum.Random;

operation Balanced(qubits : Qubit[], target : Qubit, n : Int) : Unit {
    mutable bStr = new Int[n];
    for i in 0..n-1 {
        set bStr w/= i <- DrawRandomInt(0, 1);
    }
    for i in 0..n-1 {
        if bStr[i] == 1 {
            X(qubits[i]);
        }
    }
    for qubit in qubits {
        CNOT(qubit, target);
    }
    for i in 0..n-1 {
        if bStr[i] == 1 {
            X(qubits[i]);
        }
    }
}

operation Constant(target : Qubit) : Unit {
    if DrawRandomInt(0, 1) == 1 {
        X(target);
    }
}

operation DeutschJosza(constant : Bool, n : Int) : Bool {
    use qubits = Qubit[n];
    use target = Qubit();

    X(target);
    H(target);

    within {
        ApplyToEachA(H, qubits);
    } apply {
        if constant {
            Constant(target);
        } else {
            Balanced(qubits, target, n);
        }
    }

    let resultArray = ForEach(MResetZ, qubits);
    Reset(target);

    return All(IsResultZero, resultArray);
}

In [ ]:
import time
def calculate_time(n):
        start = time.time()
        result = DeutschJosza.simulate(constant=False, n=n)
        end = time.time()
        return end - start, result

sum_times = [0] * 30
for i in range(5):
    times = []
    for n in range(1, 31):
        print(f"for {n + 1} qubits:")
        try:
            elapsed_time, result = calculate_time(n)
            times.append(elapsed_time)
            print(f"took {elapsed_time}s")
        except Exception as e:
            print(str(e))
            break
    for j in range(30):
        sum_times[j] += times[j]
avg_times = [t / 5 for t in sum_times]
print(avg_times)